In [ ]:
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 2

# imports
import collections
import functools
import io
import math
import itertools
import os
import shutil
import pathlib
import inspect
import random
import subprocess
import sys
import warnings
from pathlib import Path
import re
import json

import dipy
import dipy.core
import dipy.reconst
import dipy.reconst.dti
import dipy.segment.mask
import dipy.viz
import dotenv

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import mpl_toolkits
import matplotlib.pyplot as plt
import seaborn as sns

# Data management libraries.
import nibabel as nib
import pandas as pd

import box
from box import Box
import pprint
from pprint import pprint as ppr

# Computation & ML libraries.
import numpy as np
import skimage
import torch

import pitn

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, edgeitems=2, threshold=100, linewidth=88)
torch.set_printoptions(
    sci_mode=False, edgeitems=2, threshold=100, linewidth=88, profile="short"
)

In [ ]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = f"direnv exec {os.getcwd()} /usr/bin/env"
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [ ]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():
    # GPU information
    try:
        gpu_info = pitn.utils.system.get_gpu_specs()
        print(gpu_info)
    except NameError:
        print("CUDA Version: ", torch.version.cuda)
else:
    print("CUDA not in use, falling back to CPU")

In [ ]:
# cap is defined in an ipython magic command
print(cap)

## File and Folder Selection

In [ ]:
# Set up directories
data_dir = pathlib.Path(os.environ["DATA_DIR"])
assert data_dir.exists()
write_data_dir = pathlib.Path(os.environ["WRITE_DATA_DIR"])
assert write_data_dir.exists()
results_dir = pathlib.Path(os.environ["RESULTS_DIR"])
assert results_dir.exists()
tmp_results_dir = pathlib.Path(os.environ["TMP_RESULTS_DIR"])
assert tmp_results_dir.exists()

In [ ]:
selected_run_names = [
    "2022-02-27T16_04_20__dti_const_lr_2.5e-4_mid_net",
    "2022-02-27T03_24_34__const_lr_2.5e-4_mid_net_grad_2norm_clip_.025",
    "2022-03-01T00_33_44__tanno_espcn_baseline",
]
run_model_names = {
    "2022-02-27T16_04_20__dti_const_lr_2.5e-4_mid_net": "DIQT\nDTI",
    "2022-02-27T03_24_34__const_lr_2.5e-4_mid_net_grad_2norm_clip_.025": "DIQT\nLE",
    "2022-03-01T00_33_44__tanno_espcn_baseline": "ESPCN\nBase",
}
selected_dirs = [results_dir / d for d in selected_run_names]
assert all([d.exists for d in selected_dirs])

In [ ]:
# Load relevant result data.
runs = dict()
for name, d in zip(selected_run_names, selected_dirs):
    run_info = Box()
    run_info.test_perf = pd.read_csv(d / "test_loss.csv")
    name_col = pd.DataFrame.from_dict(
        {"run_name": list(itertools.repeat(name, len(run_info.test_perf)))}
    )
    run_info.test_perf = pd.concat([run_info.test_perf, name_col], axis=1)
    run_info.preds_file = d / "predicted_dti.zip"
    runs[name] = run_info

## Test Performance Distributions

In [ ]:
perf_metrics = (
    "rmse",
    # "nrmse",
    "log_euclid_rmse",
    # "log_euclid_nrmse",
    # "scaled_psnr",
    "ssim_fa",
    "rmse_fa",
    # "nrmse_fa"
)

perf_comparison_directions = {
    "rmse": "↓",
    # "nrmse": "↓",
    "log_euclid_rmse": "↓",
    # "log_euclid_nrmse": "↓",
    # "scaled_psnr": "↑",
    "ssim_fa": "↑",
    "rmse_fa": "↓",
    # "nrmse_fa": "↓",
}

In [ ]:
test_results = pd.concat([r.test_perf for r in runs.values()], ignore_index=True)
test_results.model.unique()

In [ ]:
test_results = pd.concat([r.test_perf for r in runs.values()], ignore_index=True)
for run, model_name in run_model_names.items():
    print(run, model_name)
    test_results.model[
        (test_results.run_name == run) & (test_results.model == "diqt")
    ] = model_name
test_results = test_results.replace(
    {"model": "cubic_spline"}, {"model": "Cubic\nSpline"}
)

In [ ]:
with mpl.rc_context(
    {
        "font.size": 10.0,
    }
):
    fig, axs = plt.subplots(
        # ncols=len(perf_metrics),
        ncols=2,
        nrows=2,
        # sharex=True,
        figsize=(6, 6),
        dpi=130,
        # gridspec_kw={"wspace": 1.0, "hspace": 1.0},
    )
    axs = axs.flatten()
    sns.despine(fig=fig, top=True, right=True)

    all_colors = sns.color_palette(
        "tab10",
        n_colors=len(test_results.model.unique()) + len(test_results.run_name.unique()),
    )
    model_colors = all_colors[: len(test_results.model.unique())]
    run_colors = all_colors[len(test_results.model.unique()) :]
    run_order = list(test_results.run_name.unique())
    
    ax_count = 0
    for i, l in enumerate(perf_metrics):

        ax = axs[i]
        df = test_results.loc[test_results.metric == l]

        vplot = sns.violinplot(
            x="model",
            y="value",
            data=df,
            ax=ax,
            scale="count",
            inner=None,
            palette=model_colors,
        )
        ax.grid(axis="y", alpha=0.5)

        points_plot = sns.swarmplot(
            x="model",
            y="value",
            # hue="run_name",
            # hue_order=run_order,
            data=df,
            ax=ax,
            # palette=run_colors,
            # palette=plt.get_cmap('gist_rainbow'),
            color="black",
            edgecolor="white",
            size=2.7,
            linewidth=0.4,
        )
        # points_plot.get_legend().remove()

        # Calculate mean performance score.
        means = df.groupby("model").mean()
        # Make sure the order follows seaborn's x-axis ordering.
        model_order = list(map(lambda ax: ax.get_text(), axs[i].get_xticklabels()))
        means = means.reindex(model_order)

        lines = ax.hlines(
            y=means.value,
            xmin=np.arange(0, len(means)) - 0.5 + 0.05,
            xmax=np.arange(1, len(means) + 1) - 0.5 - 0.05,
            colors=model_colors,
            lw=1.5,
        )

        outline_path_effects = [
            mpl.patheffects.Stroke(linewidth=5, foreground="white", alpha=0.9),
            mpl.patheffects.Normal(),
        ]
        lines.set_path_effects(outline_path_effects)

        ax.set_xticklabels(ax.get_xticklabels(), rotation=25)

        fig.canvas.draw()
        ax_format = ax.get_yaxis().get_major_formatter()

        for m, c in zip(means.value, model_colors):

            ax.annotate(
                f"{m:.4g}",
                xy=(ax.get_xlim()[0] + (ax.get_xlim()[0] * 0.4), m),
                xycoords="data",
                color=c,
                ha="right",
                va="center",
                annotation_clip=False,
                fontweight="bold",
                snap=True,
                bbox=dict(
                    boxstyle="square,pad=0.3", fc="white", lw=0, snap=True, alpha=0.75
                ),
            )
        ax.set_title(f"{l.replace('_', ' ')} {perf_comparison_directions[l]}")
        ax.set_ylabel("")
        ax.set_xlabel("")
        ax_count += 1
    for ax in axs[ax_count:]:
        sns.despine(ax=ax, left=True, bottom=True, top=True, right=True, trim=True)
        ax.set_yticks([])
        ax.set_xticks([])
        ax.set_yticklabels([])
        ax.set_xticklabels([])
plt.savefig('test_results_violin.pdf')